__<font size=5><center>Projeto Integrador - Sistema de Clusterização de Clientes para CRM</center></font>__

__Author__: Rui Ferreira Póvoa

__School__: Digital House Brazil

__Class__: Data Science

__Due Date__:   29/11/2019

<a name="1"></a>
# 1. Data Loading

<a name="1.1"></a>
## 1.1. Importação de Bibliotecas

In [1]:
#bibliotecas para análise de dados:
import pandas as pd                                        # tables and data manipulations
import numpy as np                                         # vectors and matrices

In [2]:
#bibliotecas para ler o arquivo e realizar análises de tipo de csv
import csv
import itertools                                           # funções úteis

In [3]:
#import of heap algoryth for sorting
import heapq

In [4]:
#biblioteca para tratar dimensão tempo
from datetime import date, datetime, timedelta

today_date = date(2019,10,31) #date.today()

In [5]:
#Ler dados do SQLite
import sqlite3

db = sqlite3.connect('data/vendas.db') # Opens connection if exists, else creates the database file.
conn = db.cursor() # This object lets us actually send messages to our DB and receive results.

## 1.2. Carregando os arquivos

In [6]:
filepath = "data\Pedidos.csv"

In [7]:
#Trazendo o cabeçalho:
with open(filepath, 'r', encoding = 'utf8') as csvfile:
    #head = ''.join(itertools.islice(csvfile, 1))
    #print(head)
    reader = csv.reader(csvfile)
    lst_header = next(reader)
    print(lst_header)

['id_pedido', 'id_carrinho', 'status', 'desc_status', 'id_cliente', 'id_distribuidor', 'anomes_carrinho', 'data_carrinho', 'anomes_status_pedido', 'data_status_pedido', 'num_sku', 'volumepedidos', 'volumecompras', 'volumependente', 'volumecanceladorecusado']


In [8]:
dtype_list = [np.int32,np.int32,np.int16,'str',np.int32,
              np.int32,np.int16,'str','Int32','str',
              np.int32,np.float32,np.float32,np.float32,np.float32]
dtype_dict = dict(zip(lst_header, dtype_list))
parse_dates = ['data_carrinho', 'data_status_pedido']

df_pedidos = pd.read_csv(filepath,
                 sep=',',
                 header=0,
                 encoding="utf8",
                 dtype = dtype_dict,
                 parse_dates=parse_dates)

# 2. Data Pre-processing

## 2.1 Dimensionalidade do DataFrame

In [9]:
df_pedidos.shape

(78817, 15)

## 2.2. Visualização primária dos dados

In [10]:
df_pedidos.head(5)

,id_pedido,id_carrinho,status,desc_status,id_cliente,id_distribuidor,anomes_carrinho,data_carrinho,anomes_status_pedido,data_status_pedido,num_sku,volumepedidos,volumecompras,volumependente,volumecanceladorecusado
0,3,3,0,Carrinho Abandonado,3,15,4903,2015-11-11 20:28:39.699416+00:00,NaN,NaT,0,22.900000,NaN,NaN,NaN
1,4,4,3,Cancelado/Recusado,3,15,4903,2015-11-11 20:33:45.601018+00:00,201511,2015-11-11 20:33:45.601018+00:00,0,339.000000,NaN,NaN,339.0
2,6,6,0,Carrinho Abandonado,3,15,4903,2015-11-11 20:38:16.294469+00:00,NaN,NaT,0,29890.000000,NaN,NaN,NaN
3,18,23,0,Carrinho Abandonado,12,15,4903,2015-11-12 17:13:01.843089+00:00,NaN,NaT,0,283.600006,NaN,NaN,NaN
4,28,34,0,Carrinho Abandonado,3,15,4903,2015-11-12 18:27:04.914345+00:00,NaN,NaT,0,71.900002,NaN,NaN,NaN


In [11]:
df_pedidos.dtypes.value_counts()

int32                  5
float32                4
datetime64[ns, UTC]    2
int16                  2
object                 1
Int32                  1
dtype: int64

In [12]:
df_pedidos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78817 entries, 0 to 78816
Data columns (total 15 columns):
id_pedido                  78817 non-null int32
id_carrinho                78817 non-null int32
status                     78817 non-null int16
desc_status                78817 non-null object
id_cliente                 78817 non-null int32
id_distribuidor            78817 non-null int32
anomes_carrinho            78817 non-null int16
data_carrinho              78817 non-null datetime64[ns, UTC]
anomes_status_pedido       8316 non-null Int32
data_status_pedido         8316 non-null datetime64[ns, UTC]
num_sku                    78817 non-null int32
volumepedidos              78646 non-null float32
volumecompras              5066 non-null float32
volumependente             144 non-null float32
volumecanceladorecusado    3104 non-null float32
dtypes: Int32(1), datetime64[ns, UTC](2), float32(4), int16(2), int32(5), object(1)
memory usage: 5.2+ MB


## 2.4. Dados em Falta (NaN)

In [13]:
def missing_data(data):
    #transformando os dados num objecto Series
    total = data.isnull().sum()
    
    # calculando a percentagem por coluna
    percent = round(data.isnull().sum() / len(data) * 100, 2)
    
    #A função concat coloca uma Series/Dataframe a seguir ao outro segundo o eixo que se quiser
    tt = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    
    #colcando o data types das colunas numa lista
    types = []
    for col in data.columns:
        dtype = str(data[col].dtype)
        types.append(dtype)
    tt['Types'] = types
    return(np.transpose(tt))

In [14]:
%%time
missing_data(df_pedidos)

Wall time: 150 ms


,id_pedido,id_carrinho,status,desc_status,id_cliente,id_distribuidor,anomes_carrinho,data_carrinho,anomes_status_pedido,data_status_pedido,num_sku,volumepedidos,volumecompras,volumependente,volumecanceladorecusado
Total,0,0,0,0,0,0,0,0,70501,70501,0,171,73751,78673,75713
Percent,0,0,0,0,0,0,0,0,89.45,89.45,0,0.22,93.57,99.82,96.06
Types,int32,int32,int16,object,int32,int32,int16,"datetime64[ns, UTC]",Int32,"datetime64[ns, UTC]",int32,float32,float32,float32,float32


In [15]:
df_pedidos.desc_status.value_counts()

Carrinho Abandonado    70501
Aprovado                5066
Cancelado/Recusado      3106
Pendente                 144
Name: desc_status, dtype: int64

# 3. Feature Engineering
## 3.1. Super Features

In [16]:
def f(x):
    d = {}
    
    #d['pedidos'] = x['id_pedido'].count()
    d['pedidos_efetivos'] = x['volumecompras'].count()
    #d['pedidos_cancelados'] = x['volumecanceladorecusado'].count()
    #d['volume_compras'] = x['volumecompras'].sum()
    #d['volume_cancelados'] = x['volumecanceladorecusado'].sum()
    
    d['max_pedido'] = x['volumecompras'].max()
    d['min_pedido'] = x['volumecompras'].min()
    d['tk_med_pedido'] =  x['volumecompras'].mean()
    d['desvio_padrao_pedido'] = x['volumecompras'].std()
    #d['dias'] = (today_date - x['data_carrinho'].min().date()).days / 365.25 #transformando em ano
    d['tk_ano'] = x['volumecompras'].sum() / ((today_date - x['data_carrinho'].min().date()).days / 365.25)
    d['pedidos_ano'] = x['volumecompras'].count() / ((today_date - x['data_carrinho'].min().date()).days / 365.25)
    d['data_cadastro'] = (today_date - x['data_carrinho'].min().date()).days
    d['data_ult_pedido'] = (today_date - x['data_status_pedido'].max().date()).days
     
    d['frequencia_med'] = (x['data_status_pedido'] - x['data_status_pedido'].shift(1)).mean() / timedelta(days=1)
    d['frequencia_std'] = (x['data_status_pedido'] - x['data_status_pedido'].shift(1)).std() / np.timedelta64(1, 'D')
    d['recencia_max'] = (x['data_status_pedido'] - x['data_status_pedido'].shift(1)).max() / np.timedelta64(1, 'D')
    
    return pd.Series(d, index=['pedidos_efetivos','max_pedido','min_pedido',
                               'tk_med_pedido','desvio_padrao_pedido',
                               'tk_ano', 'pedidos_ano',
                               'data_cadastro','data_ult_pedido',
                               'frequencia_med','frequencia_std',
                               'recencia_max'])

df_pedidos.sort_values(by='data_status_pedido', 
                       ascending = True).groupby(by='id_cliente').apply(f).sort_values(by='pedidos_efetivos', 
                                                                                       ascending = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


,pedidos_efetivos,max_pedido,min_pedido,tk_med_pedido,desvio_padrao_pedido,tk_ano,pedidos_ano,data_cadastro,data_ult_pedido,frequencia_med,frequencia_std,recencia_max
id_cliente,,,,,,,,,,,,
3,1299.0,2520.000000,50.500000,498.963867,355.184143,163267.883534,327.213621,1450.0,3.0,1.050678,2.810389,55.016411
1072,242.0,4252.399902,111.599998,769.517883,753.504761,63038.079443,81.918906,1079.0,2.0,4.137238,10.152554,53.610441
1456,237.0,6852.000000,191.250000,1699.367920,1080.818848,146664.432788,86.305334,1003.0,7.0,3.427059,6.708412,56.178038
130,133.0,4469.799805,201.630005,880.992554,888.462769,32446.618742,36.829606,1319.0,261.0,7.666422,25.628239,294.976786
4211,117.0,2934.239990,200.160004,678.845032,401.272797,47170.622342,69.486585,615.0,33.0,4.809147,6.982436,42.893191
...,...,...,...,...,...,...,...,...,...,...,...,...
69700,0.0,NaN,NaN,NaN,NaN,0.000000,0.000000,248.0,NaN,NaN,NaN,NaN
69707,0.0,NaN,NaN,NaN,NaN,0.000000,0.000000,248.0,NaN,NaN,NaN,NaN
69723,0.0,NaN,NaN,NaN,NaN,0.000000,0.000000,248.0,NaN,NaN,NaN,NaN


In [17]:
df_FE = df_pedidos.sort_values(by='data_status_pedido', 
                       ascending = True).groupby(by='id_cliente').apply(f).sort_values(by='pedidos_efetivos', 
                                                                                       ascending = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [18]:
df_FE.dropna(axis=0, subset=['max_pedido'], inplace = True)

In [19]:
df_FE.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 698 entries, 3 to 525972
Data columns (total 12 columns):
pedidos_efetivos        698 non-null float64
max_pedido              698 non-null float64
min_pedido              698 non-null float64
tk_med_pedido           698 non-null float64
desvio_padrao_pedido    370 non-null float64
tk_ano                  698 non-null float64
pedidos_ano             698 non-null float64
data_cadastro           698 non-null float64
data_ult_pedido         698 non-null float64
frequencia_med          490 non-null float64
frequencia_std          368 non-null float64
recencia_max            490 non-null float64
dtypes: float64(12)
memory usage: 70.9 KB


In [20]:
df_FE

,pedidos_efetivos,max_pedido,min_pedido,tk_med_pedido,desvio_padrao_pedido,tk_ano,pedidos_ano,data_cadastro,data_ult_pedido,frequencia_med,frequencia_std,recencia_max
id_cliente,,,,,,,,,,,,
3,1299.0,2520.000000,50.500000,498.963867,355.184143,163267.883534,327.213621,1450.0,3.0,1.050678,2.810389,55.016411
1072,242.0,4252.399902,111.599998,769.517883,753.504761,63038.079443,81.918906,1079.0,2.0,4.137238,10.152554,53.610441
1456,237.0,6852.000000,191.250000,1699.367920,1080.818848,146664.432788,86.305334,1003.0,7.0,3.427059,6.708412,56.178038
130,133.0,4469.799805,201.630005,880.992554,888.462769,32446.618742,36.829606,1319.0,261.0,7.666422,25.628239,294.976786
4211,117.0,2934.239990,200.160004,678.845032,401.272797,47170.622342,69.486585,615.0,33.0,4.809147,6.982436,42.893191
...,...,...,...,...,...,...,...,...,...,...,...,...
55296,1.0,1127.210449,1127.210449,1127.210449,NaN,1502.604440,1.333029,274.0,269.0,NaN,NaN,NaN
54279,1.0,175.520004,175.520004,175.520004,NaN,233.122478,1.328182,275.0,275.0,0.003724,NaN,0.003724
1614107,1.0,620.809204,620.809204,620.809204,NaN,45350.112360,73.050000,5.0,4.0,NaN,NaN,NaN


## 3.2. Features de Categoria de Produto

In [21]:
query = """
SELECT id_cliente, categoria FROM(
	SELECT	
		id_cliente,
		d.parent_id categoria,
		--CASE WHEN e.tipo = 'PF' THEN 0 ELSE 1 END is_PJ,
		Sum(volumecompras) / count(volumecompras) 	tk_med,
		RANK() OVER (
			PARTITION BY id_cliente
			ORDER BY Sum(volumecompras) / count(volumecompras) DESC
		) rank_cat
	FROM VENDAS a
	left join sku_sku2 b
		on a.id_sku = b.id
	left join consumidores e
		on id_cliente = e.user_id
	left join category_category c
		ON B.category_id = c.id
	left join category_category d
		ON c.parent_id = d.id
	WHERE id_sku is not NULL and d.parent_id is not NULL
	group by
	id_cliente,
	d.parent_id
	--CASE WHEN e.tipo = 'PF' THEN 0 ELSE 1 END
    )
where rank_cat = 1 and tk_med is not NULL
"""

df_categoria = pd.read_sql_query(query,db,index_col = 'id_cliente')

In [22]:
df_categoria

,categoria
id_cliente,
3,378
5,378
19,378
103,65
120,54
...,...
1610061,65
1610779,65
1611459,147


In [23]:
df_categoria.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 636 entries, 3 to 1614107
Data columns (total 1 columns):
categoria    636 non-null int64
dtypes: int64(1)
memory usage: 9.9 KB


In [24]:
pd.get_dummies(df_categoria['categoria'])

,18,46,54,58,65,99,147,255,325,378
id_cliente,,,,,,,,,,
3,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,1
19,0,0,0,0,0,0,0,0,0,1
103,0,0,0,0,1,0,0,0,0,0
120,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1610061,0,0,0,0,1,0,0,0,0,0
1610779,0,0,0,0,1,0,0,0,0,0
1611459,0,0,0,0,0,0,1,0,0,0


In [25]:
df_cat_dummies = pd.get_dummies(df_categoria['categoria'])

In [26]:
df_merged = df_FE.merge(df_cat_dummies,
                       how='left', 
                       on='id_cliente')

In [27]:
df_merged

,pedidos_efetivos,max_pedido,min_pedido,tk_med_pedido,desvio_padrao_pedido,tk_ano,pedidos_ano,data_cadastro,data_ult_pedido,frequencia_med,...,18,46,54,58,65,99,147,255,325,378
id_cliente,,,,,,,,,,,,,,,,,,,,,
3,1299.0,2520.000000,50.500000,498.963867,355.184143,163267.883534,327.213621,1450.0,3.0,1.050678,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1072,242.0,4252.399902,111.599998,769.517883,753.504761,63038.079443,81.918906,1079.0,2.0,4.137238,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1456,237.0,6852.000000,191.250000,1699.367920,1080.818848,146664.432788,86.305334,1003.0,7.0,3.427059,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
130,133.0,4469.799805,201.630005,880.992554,888.462769,32446.618742,36.829606,1319.0,261.0,7.666422,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4211,117.0,2934.239990,200.160004,678.845032,401.272797,47170.622342,69.486585,615.0,33.0,4.809147,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55296,1.0,1127.210449,1127.210449,1127.210449,NaN,1502.604440,1.333029,274.0,269.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
54279,1.0,175.520004,175.520004,175.520004,NaN,233.122478,1.328182,275.0,275.0,0.003724,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1614107,1.0,620.809204,620.809204,620.809204,NaN,45350.112360,73.050000,5.0,4.0,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [28]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 698 entries, 3 to 525972
Data columns (total 22 columns):
pedidos_efetivos        698 non-null float64
max_pedido              698 non-null float64
min_pedido              698 non-null float64
tk_med_pedido           698 non-null float64
desvio_padrao_pedido    370 non-null float64
tk_ano                  698 non-null float64
pedidos_ano             698 non-null float64
data_cadastro           698 non-null float64
data_ult_pedido         698 non-null float64
frequencia_med          490 non-null float64
frequencia_std          368 non-null float64
recencia_max            490 non-null float64
18                      636 non-null float64
46                      636 non-null float64
54                      636 non-null float64
58                      636 non-null float64
65                      636 non-null float64
99                      636 non-null float64
147                     636 non-null float64
255                     636 non-null 

In [ ]:
#Faltou só  tratar os nulos!! para as dummies basta preencher tudo com zero e depois transforma-las em integer!

#Para desvio padrão não sei bem o que fazer porque são os casos que só tem um resultado.. então 0

#A recencia e frequencia pode ser preenchido com a data-ult-pedido uma vez que só tem um pedido?
#tentei mas deu erro

## 3.3. Trantando os Vazios

In [29]:
df_merged[['desvio_padrao_pedido',
           'frequencia_std',
           18, 46, 54, 
           58, 65, 99,
           147,255,325,378]] = df_merged[['desvio_padrao_pedido',
                                                  'frequencia_std',18, 46, 54,
                                          58, 65, 99,
                                          147,255,325,378]].fillna(value=0)

In [30]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 698 entries, 3 to 525972
Data columns (total 22 columns):
pedidos_efetivos        698 non-null float64
max_pedido              698 non-null float64
min_pedido              698 non-null float64
tk_med_pedido           698 non-null float64
desvio_padrao_pedido    698 non-null float64
tk_ano                  698 non-null float64
pedidos_ano             698 non-null float64
data_cadastro           698 non-null float64
data_ult_pedido         698 non-null float64
frequencia_med          490 non-null float64
frequencia_std          698 non-null float64
recencia_max            490 non-null float64
18                      698 non-null float64
46                      698 non-null float64
54                      698 non-null float64
58                      698 non-null float64
65                      698 non-null float64
99                      698 non-null float64
147                     698 non-null float64
255                     698 non-null 